In [1]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Урок 3. Embedding word2vec fasttext

Задача поиск похожих по эмбеддингам

Скачиваем датасет (источник): положительные, отрицательные.

или можно через ноутбук

!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

что надо сделать
1. объединить в одну выборку

In [4]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2022-06-09 14:49:47--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-09 14:49:47--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd5d974f479f26199fb17bedc12.dl.dropboxusercontent.com/cd/0/inline/Bm4DFp8Tw6RFDhFYZtCYUcivmvU72Tz1SS4CaNnrHqUrGrykIn0hBe6AyXcv_9hC4r3zlLD6kP2clqBaMg99qjrXEzIjRt1eVTskS6OlvFLK4jqHiXL_2TwJglO4kmJ-OE6aBXDXMipAFXSK_1WjXPpU4VljVppw8a1qMexib6CQRg/file# [following]
--2022-06-09 14:49:47--  https://ucd5d974f479f26199fb17bedc12.dl.dropboxusercontent.com/cd/0/inline/Bm4DFp8Tw6RFDhFYZtCYUcivmvU72Tz1SS4CaNnrHqUrGrykIn0hBe6AyXcv_9hC4

In [5]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

In [6]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])

df = positive.append(negative)

df = df.reset_index().drop('index', axis=1)

In [7]:
df.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [8]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0


for line in df['text']:
    spls = preprocess_txt(line)
    sentences.append(spls)

In [10]:
sentences = [i for i in sentences if len(i) > 2]

In [11]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=1)

In [12]:
modelFT = FastText(sentences=sentences, size=300, min_count=1, window=5, workers=8)

In [13]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in tqdm_notebook(df['text']):
    n_w2v = 0
    n_ft = 0
    spls = line.split("\t")
    #index_map[counter] = spls[1]
    question = preprocess_txt(spls[0])
    
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

    counter += 1

w2v_index.build(10)
ft_index.build(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/226834 [00:00<?, ?it/s]

True

In [21]:
def get_response(question, index, model):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5 )
    return answers

In [22]:
df['text'][0]

'@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'

In [27]:
get_response(df['text'][0], w2v_index, modelW2V)

[0, 87109, 161595, 37404, 7704]

In [26]:
df.loc[get_response(df['text'][0], w2v_index, modelW2V), 'text']

0         @first_timee хоть я и школота, но поверь, у на...
87109     @mad_bad_bitch чё там то самое оно? ты их случ...
161595     @ushikovaa @victoria75132 о_О тот самый сок? :-D
37404     всего самая первая фраза, которую мы узнали на...
7704      только Алла могла за время игры выдать фразу "...
Name: text, dtype: object

In [29]:
df.loc[get_response(df['text'][0], w2v_index, modelFT), 'text']

174222        RT @motherfuckersgo: @11DoctorAreCool нубля((
66883     RT @Katrina0298: @Antil_opa @Wahnsinn_Ww @oksa...
115448    RT @linapanova0: @VMeysters ну итж издивательс...
158803    RT @raduznazebra: @orange_tie тысячник зажравш...
176879       RT @H_A_P_P_I_L_Y: @XA33_HA_XA33E почиму?(((((
Name: text, dtype: object